In [ ]:
#모듈
import math
import pandas as pd
import numpy as np

try:
    import cufflinks as cf
except:
    !pip install cufflinks
    import cufflinks as cf

cf.go_offline()

from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor,BaggingRegressor
from sklearn.neural_network import MLPRegressor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 다루기
pd.set_option("Display.max_columns",200)
import re
import time

# 데이터 시각화
import seaborn as sns

# 사용할 머신러닝 모델들 - RandomForest, AdaBoost, GradientBoost, ExtraTrees
import sklearn
from sklearn.metrics import log_loss
from sklearn.svm import SVC

In [ ]:
#명목형 변수 더미화
def dummify_category_cols(df,cols):
    dummies = []
    new_df = df.copy()

    for idx in range(len(cols)):
        new_df = new_df.join(pd.get_dummies(total[cols[idx]], prefix = cols[idx]))
        new_df = new_df.drop(cols[idx], axis =1)
    return new_df

In [ ]:
#연속형 변수 범주화
def dummify_number_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: int(x)//num * num)
    return new_df


In [ ]:
#명목형 변수의 연속형 변환 - 변수의 차원을 낮추기 위하여
def num_category(df, col, chg) :
    new_df = df.copy()
    set = {}
    
    for idx in range(len(chg)) :
        set[chg[idx]] = idx
        
    new_df[col] = new_df[col].map(set)
    
    return new_df        

In [ ]:
#숫자형 변수 일정 이상은 묶어버리기
def number_group_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: num if x> num else x )
    return new_df


In [ ]:
#RMS 구하기
def RMSE(y_true,y_pred):
    
    squaredErrors = (y_pred - y_true)**2
    MSE = squaredErrors.mean()
    
    return math.sqrt(MSE)

In [ ]:
#본격적으로 시작

In [ ]:
training = pd.read_csv("house_price.csv")
test = pd.read_csv("test.csv")


In [ ]:
#training = training[training.GrLivArea <= 5600]
#training = training[training["1stFlrSF"] <= 4600]
#training = training[training.TotalBsmtSF < 6000]
#training = training[training.MasVnrArea <= 1600]

In [ ]:
total = pd.concat([training, test]).reset_index()
del total["index"]

In [ ]:
total.head(2)

In [ ]:
total.columns

In [ ]:
#선택된 변수
category_selected = ["MSSubClass", "MSZoning", "Street" ,"LandContour", "LotConfig", 
            "Neighborhood", "BldgType", "HouseStyle", "OverallQual", 
            "MasVnrType", "ExterQual", "ExterCond",
            "Foundation","BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC", "CentralAir",
            "Electrical", "BsmtFullBath", "FullBath", "HalfBath", "KitchenQual",
            "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageType",
            "GarageCars", "PavedDrive", "SaleCondition", "GarageFinish"]

extra_selected = [ "GarageCars", "Electrical" ,"OverallQual","OverallCond","ExterQual", "ExterCond",  "BsmtQual",
                  "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC","KitchenQual","FireplaceQu", "Functional"]

number_selected = ["GarageYrBlt","YearBuilt", "YearRemodAdd", "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF", 
                   "1stFlrSF", "2ndFlrSF", "GrLivArea", 'GarageArea', "LotArea", "MasVnrArea", "WoodDeckSF", "OpenPorchSF"]

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [ ]:
#선택된 변수
category_selected = ['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'Id', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'PoolQC', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'ScreenPorch', 'Street',
       'TotRmsAbvGrd', 'TotalBsmtSF', 'Utilities', 'WoodDeckSF', 'YearBuilt',
       'YearRemodAdd', 'YrSold']

extra_selected = [ "GarageCars", "Electrical" ,"OverallQual","OverallCond","ExterQual", "ExterCond",  "BsmtQual",
                  "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC","KitchenQual","FireplaceQu", "Functional"]

number_selected =['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea','GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
category_selected = [x for x in category_selected if x not in number_selected]

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [ ]:
new_total = total[selected_with_SalePrice].copy()
new_total.head(2)

In [ ]:
#임시 전처리

In [ ]:
new_total.head()

In [ ]:
#전처리1 - 연속형데이터 중 null값이 있는 행 모두 버려주기
new_total[number_selected].isnull().sum()

In [ ]:
new_total["YearBuilt"] = new_total["YearBuilt"].fillna(new_total.YearBuilt.mean())

In [ ]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].fillna(new_total.BsmtFinSF1.mean())

In [ ]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].fillna(new_total.BsmtUnfSF.mean())

In [ ]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].fillna(new_total.TotalBsmtSF.mean())

In [ ]:
new_total["GarageArea"] = new_total["GarageArea"].fillna(new_total.GarageArea.mean())

In [ ]:
new_total["GarageYrBlt"] = new_total["GarageYrBlt"].fillna(new_total.GarageYrBlt.mean())

In [ ]:
new_total["Electrical"] = new_total["Electrical"].fillna("SBrkr")

In [ ]:
new_total = new_total.fillna("404")

In [ ]:
new_total.isnull().sum()

In [ ]:
new_total.head()

In [ ]:
#명목형 변수를 연속형으로 바꿔주는 것

In [ ]:
new_total = num_category(new_total, "ExterQual", ["Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "ExterCond", ["Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "BsmtQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "BsmtCond", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "BsmtExposure", ["404", "No", "Fa", "Mn", "Av", "Gd"])

In [ ]:
new_total = num_category(new_total, "BsmtFinType1", ["404", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"])

In [ ]:
new_total = num_category(new_total, "HeatingQC", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "KitchenQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "FireplaceQu", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "Electrical", ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"])

In [ ]:
new_total = num_category(new_total, "Functional", ["404", "Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"])
new_total.Functional.unique()

In [ ]:
#변수끼리 합쳐주는 것

In [ ]:
#new_total["Functional"] = total["Functional"].map(lambda x : 1 if x==8 else 0)
#new_total["LotConfig"] = new_total.LotConfig.map(lambda x : 1 if x == "CulDSac" else 0)
#new_total["FullBath"] = new_total.FullBath.map(lambda x : 1 if x <=2 else x-1)
#new_total["OverallCond"] = new_total.OverallCond.map(lambda x : 1 if x <=4 else 2)
#new_total["BldgType"] = new_total.BldgType.map(lambda x : 1 if x =="1Fam" else 2)
#new_total = number_group_cols(new_total, ["TotRmsAbvGrd"], [13])
#new_total = number_group_cols(new_total, ["HalfBath"], [1])
#new_total = number_group_cols(new_total, ["FullBath"], [2])

In [ ]:
#임시 전처리

In [ ]:
new_total = new_total.assign(Is_Remod =  lambda x : x["YearBuilt"] != x["YearRemodAdd"])
new_total["Is_Remod"] = new_total.YearRemodAdd.map(lambda x : 1 if x == True else 0)
new_total["after"] = 2011 - new_total["YearRemodAdd"]
new_total = new_total.drop("YearRemodAdd", axis = 1)

In [ ]:
#new_total = new_total.assign(Is_2nd =  lambda x : x["2ndFlrSF"] != 0)
#new_total["Is_2nd"] = new_total.Is_2nd.map(lambda x : 1 if x == True else 0)

In [ ]:
#new_total = new_total.assign(Is_bsmt_unfinish = lambda x : x["BsmtFinType1"] == 0)
#new_total["Is_bsmt_unfinish"] = new_total.Is_bsmt_unfinish.map(lambda x : 1 if x == True else 0)

In [ ]:
#new_total = new_total.assign(Is_Fireplaces = lambda x : x["Fireplaces"] != 0)
#new_total["Is_Fireplaces"] = new_total.Is_Fireplaces.map(lambda x : 1 if x == True else 0)

In [ ]:
#new_total = new_total.assign(Is_GarageFinish = lambda x : x["GarageFinish"] != "Unf")
#new_total["Is_GarageFinish "] = new_total.Is_GarageFinish .map(lambda x : 1 if x == True else 0)

In [ ]:
new_total["total_qual"] = (new_total['OverallQual'] + new_total["ExterQual"]+new_total["BsmtQual"]+ new_total["HeatingQC"] + 
                           new_total["KitchenQual"] + new_total["FireplaceQu"])

In [ ]:
new_total["total_point"] = (new_total['OverallQual'] + new_total["ExterQual"]+
                            new_total["BsmtQual"]+ new_total["Functional"] + 
                            new_total["Electrical"]+new_total["ExterCond"] +
                            new_total["BsmtCond"] + new_total["OverallCond"] +
                            new_total["BsmtExposure"] + new_total["HeatingQC"] + new_total["KitchenQual"] + new_total["FireplaceQu"] +
                            new_total["BsmtFinType1"])

In [ ]:
new_total["total_cond"] = (new_total["ExterCond"] +
                            new_total["BsmtCond"] + new_total["OverallCond"] )

In [ ]:
new_total["ratio_bsmt"] = new_total["BsmtUnfSF"] / new_total["TotalBsmtSF"]
new_total["ratio_bsmt"] = new_total["ratio_bsmt"].map(lambda x : x if x>=0 else 100000)

In [ ]:
new_total["Total_Area"] = new_total["GrLivArea"] + new_total["LotArea"] + new_total["GarageArea"] + new_total["TotalBsmtSF"]

In [ ]:
new_total["GrLivArea"] = new_total["GrLivArea"].map(lambda x : math.log(x))

In [ ]:
new_total["LotArea"] = new_total["LotArea"].map(lambda x : math.log(x))

In [ ]:
new_total["GarageArea"] = new_total["GarageArea"].map(lambda x : math.log(x+1))

In [ ]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].map(lambda x : math.log(x+1))

In [ ]:
new_total["Total_Area"] = new_total["Total_Area"].map(lambda x : math.log(x+1))

In [ ]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].map(lambda x : math.log(x+1))

In [ ]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].map(lambda x : math.log(x+1))

In [ ]:
new_total["1stFlrSF"] = new_total["1stFlrSF"].map(lambda x : math.log(x+1))

In [ ]:
new_total["2ndFlrSF"] = new_total["2ndFlrSF"].map(lambda x : math.log(x+1))

In [ ]:
new_total["MasVnrArea"] = new_total["MasVnrArea"].map(lambda x : math.log(int(x)+1))
new_total["WoodDeckSF"] = new_total["WoodDeckSF"].map(lambda x : math.log(int(x)+1))
new_total["OpenPorchSF"] = new_total["OpenPorchSF"].map(lambda x : math.log(int(x)+1))

In [ ]:
new_total["SalePrice"] = new_total.SalePrice.map(lambda x: math.log(int(x)+1))

In [ ]:
#확인

In [ ]:
new_total.head(2)

In [ ]:
#히트맵으로 확인
#training = training = new_total.iloc[:len(training)]

In [ ]:
#n= len(selected_with_SalePrice)
#colormap = plt.cm.viridis
#plt.figure(figsize=(n,n))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(training.corr(),
#            linewidths=0.1,
#            vmax=1.0, 
#            square=True, 
#            cmap=colormap, 
#            linecolor='white', 
#            annot=True)

In [ ]:
new_total2 = dummify_category_cols(new_total,category_selected)

In [ ]:
new_total2.head(2)

In [ ]:
training = new_total2.iloc[:len(training)]
test = new_total2.iloc[len(training):]

In [ ]:
#여기서 selected 재설정

In [ ]:
new_total.isnull().sum()

In [ ]:
training['is_train'] = np.random.uniform(0, 1, len(training)) <= .75

In [ ]:
training_set = training[training['is_train']==True]
test_set = training[training['is_train']==False]

In [ ]:
forest = RandomForestRegressor(n_estimators=6000, n_jobs=3)
#adaboost = AdaBoostRegressor(n_estimators=5000)
#gradientboosting = GradientBoostingRegressor(loss = "huber", n_estimators=5000, max_depth=15)
#extratrees = ExtraTreesRegressor(n_estimators=6000, n_jobs =3)
bagging = BaggingRegressor(n_estimators=6000,n_jobs =3 )
#mlp = MLPRegressor(hidden_layer_sizes = (150,1), activation = "relu", max_iter=100000)

In [ ]:
feature = new_total2.columns.drop("SalePrice")
target = ["SalePrice"]

In [ ]:
forest.fit(training_set[feature], training_set[target])
#adaboost.fit(training_set[feature], training_set[target])
#gradientboosting.fit(training_set[feature], training_set[target])
#extratrees.fit(training_set[feature], training_set[target])
bagging.fit(training_set[feature], training_set[target])
#mlp.fit(training_set[feature], training_set[target])


In [ ]:
#forest.predict(test_set[feature])

In [ ]:
print("forest : ",forest.score(test_set[feature], test_set[target]))
#print("adaboost : ",adaboost.score(test_set[feature], test_set[target]))
#print("gb : ",gradientboosting.score(test_set[feature], test_set[target]))
#print("extratrees : ",extratrees.score(test_set[feature], test_set[target]))
print("bagging : ",bagging.score(test_set[feature], test_set[target]))
#print("mlp : ",mlp.score(test_set[feature], test_set[target]))

In [ ]:
y_pred1=forest.predict(test_set[feature])
#y_pred2=adaboost.predict(test_set[feature])
#y_pred3=gradientboosting.predict(test_set[feature])
#y_pred4=extratrees.predict(test_set[feature])
y_pred5=bagging.predict(test_set[feature])
#y_pred6=mlp.predict(test_set[feature])

In [ ]:
y_pred1 = np.array(list(map(lambda x: math.exp(x)-1,y_pred1)))
#y_pred2 = np.array(list(map(lambda x: math.exp(x)-1,y_pred2)))
#y_pred3 = np.array(list(map(lambda x: math.exp(x)-1,y_pred3)))
#y_pred4 = np.array(list(map(lambda x: math.exp(x)-1,y_pred4)))
y_pred5 = np.array(list(map(lambda x: math.exp(x)-1,y_pred5)))
#y_pred6 = np.array(list(map(lambda x: math.exp(x)-1,y_pred6)))

In [ ]:
y_true = test_set["SalePrice"]
y_ture= np.array(list(map(lambda x: math.exp(x)-1,y_true)))

In [ ]:
rmse1 = RMSE(np.exp(y_true),y_pred1)
#rmse2 = RMSE(np.exp(y_true),y_pred2)
#rmse3 = RMSE(np.exp(y_true),y_pred3)
#rmse4 = RMSE(np.exp(y_true),y_pred4)
rmse5 = RMSE(np.exp(y_true),y_pred5)
#rmse6 = RMSE(np.exp(y_true),y_pred6)
print("forest : ",rmse1)
#print("adaboost : ",rmse2)
#print("gb : ",rmse3)
#print("extratrees : ",rmse4)
print("bagging : ",rmse5)
#print("mlp : ",rmse6)

In [ ]:
#importance = pd.DataFrame(forest.feature_importances_, index = training_set[feature].columns)

In [ ]:
#(
#    importance.assign(header = lambda x: x.index.str.split("_").str[0])
#                  .groupby("header")[0]
#                  .sum()
#                  .iplot(kind="bar")
#)

In [ ]:
#최적화

In [ ]:
best_score = 0
for N in range(4000,8500,500):
    tmp = RandomForestRegressor(n_estimators=N)
    tmp.fit(training_set[feature], training_set[target])
    score = tmp.score(test_set[feature],test_set[target])
    print("최적화 n_estimator 값 : {}".format(N))
    print("예측률 : {}".format(score))
    
    if score>best_score:
        best_score = score
        best_N = N
        best_rf_model = tmp
        
print("최적화 n_estimator 값 : {}".format(best_N))
print("예측률 : {}".format(best_score))

In [ ]:
best_score = 0
for N in range(13,21):
    tmp = RandomForestRegressor(n_estimators=4500, max_depth = N)
    tmp.fit(training_set[feature], training_set[target])
    score = tmp.score(test_set[feature],test_set[target])
    
    if score>best_score:
        best_score = score
        best_N = N
        best_rf_model = tmp
        
print("최적화 max_depth 값 : {}".format(best_N))
print("예측률 : {}".format(best_score))

In [ ]:
#제출

In [ ]:
forest.fit(training[feature], training[target])
#adaboost.fit(training[feature], training[target])
#gradientboosting.fit(training[feature], training[target])
#extratrees.fit(training[feature], training[target])
bagging.fit(training[feature], training[target])
#mlp.fit(training[feature], training[target])

In [ ]:
prediction1 =forest.predict(test[feature])
#prediction2 =adaboost.predict(test[feature])
#prediction3 =gradientboosting.predict(test[feature])
#prediction4 =extratrees.predict(test[feature])
prediction5 =bagging.predict(test[feature])
#prediction6 =mlp.predict(test[feature])

In [ ]:
prediction1 = np.array(list(map(lambda x: math.exp(x)-1,prediction1)))
#prediction2 = np.array(list(map(lambda x: math.exp(x)-1,prediction2)))
#prediction3 = np.array(list(map(lambda x: math.exp(x)-1,prediction3)))
#prediction4 = np.array(list(map(lambda x: math.exp(x)-1,prediction4)))
prediction5 = np.array(list(map(lambda x: math.exp(x)-1,prediction5)))
#prediction6 = np.array(list(map(lambda x: math.exp(x)-1,prediction6)))

In [ ]:
submission1 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction1})
#submission2 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction2})
#submission3 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction3})
#submission4 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction4})
submission5 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction5})
#submission6 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction6})

In [ ]:
submission1.to_csv("Submission_forest5.csv")
#submission2.to_csv("Submission_adaboost3.csv")
#submission3.to_csv("Submission_gradientboosting3.csv")
#submission4.to_csv("Submission_extratrees3.csv")
submission5.to_csv("Submission_bagging5.csv")
#submission6.to_csv("Submission_mlp1.csv")